# Handwritten Digit Recognition - Webcam Detection

This notebook uses the trained model to perform real-time digit recognition from your webcam.

**Instructions:**
- Run all cells to start the webcam
- Show handwritten digits (0-9) to the camera
- The model will predict the digit and display confidence
- Press 'q' to quit

## 1. Import Libraries

In [1]:
import cv2
import numpy as np
from tensorflow import keras
import os

print("Libraries imported successfully!")

d:\Versity space\Study matterials\01068-COMPUTER-VISION-AND-PATTERN-RECOGNITION\FINAL\model\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Libraries imported successfully!


## 2. Load Trained Model

In [2]:
# Path to the trained model
model_path = 'output/best_model.h5'

# Check if model exists
if not os.path.exists(model_path):
    print(f"ERROR: Model not found at {model_path}")
    print("Please run the training notebook first!")
else:
    # Load the model
    model = keras.models.load_model(model_path)
    print(f"✅ Model loaded successfully from {model_path}")
    print(f"Model input shape: {model.input_shape}")
    print(f"Model output shape: {model.output_shape}")

✅ Model loaded successfully from output/best_model.h5
Model input shape: (None, 784)
Model output shape: (None, 10)


## 3. Preprocessing Function

In [3]:
def preprocess_frame(roi):
    # Convert to grayscale
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # Reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold (white digit on black background)
    _, thresh = cv2.threshold(
        blur, 0, 255,
        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )

    # Resize to MNIST size
    resized = cv2.resize(thresh, (28, 28))

    # Normalize
    normalized = resized.astype("float32") / 255.0

    # ✅ FLATTEN FOR DENSE MODEL
    flattened = normalized.reshape(1, 784)

    return flattened, normalized


## 4. Start Webcam Detection

**Controls:**
- Press **'q'** to quit
- Press **'s'** to take a snapshot
- Show handwritten digits to the camera for recognition

In [4]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("ERROR: Cannot access webcam!")
else:
    print("Webcam started successfully!")
    print("Press 'q' to quit, 's' to save snapshot")

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    snapshot_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        display_frame = frame.copy()

        # ROI
        h, w = frame.shape[:2]
        roi_size = 200
        x1 = (w - roi_size) // 2
        y1 = (h - roi_size) // 2
        x2 = x1 + roi_size
        y2 = y1 + roi_size

        roi = frame[y1:y2, x1:x2]

        # Preprocess
        preprocessed, resized = preprocess_frame(roi)

        # Predict
        prediction = model.predict(preprocessed, verbose=0)
        digit = np.argmax(prediction)
        confidence = prediction[0][digit] * 100

        # Draw ROI
        cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Text background
        overlay = display_frame.copy()
        cv2.rectangle(overlay, (10, 10), (300, 120), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.6, display_frame, 0.4, 0, display_frame)

        # Display text
        cv2.putText(display_frame, f"Digit: {digit}", (20, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
        cv2.putText(display_frame, f"Confidence: {confidence:.1f}%", (20, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # Show preprocessed digit
        preview = cv2.resize(resized, (100, 100))
        preview = cv2.cvtColor(preview, cv2.COLOR_GRAY2BGR)
        display_frame[10:110, w-110:w-10] = (preview * 255).astype(np.uint8)
        cv2.rectangle(display_frame, (w-110, 10), (w-10, 110), (255, 255, 255), 2)

        cv2.imshow("Digit Recognition - Webcam", display_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            cv2.imwrite(f"snapshot_{snapshot_count}.png", display_frame)
            snapshot_count += 1

    cap.release()
    cv2.destroyAllWindows()


Webcam started successfully!
Press 'q' to quit, 's' to save snapshot
